In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from collections import Counter
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
# data load

store = pd.read_csv('../input/store.csv')
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
def storeinfo(store):
    # для удобства
    store = store.set_index('Store')
    
    # нужно знать конкретную дату, 3000:12 означает нет данных
    store.CompetitionOpenSinceMonth = store.CompetitionOpenSinceMonth.map(lambda x : int(np.nan_to_num(x) + np.isnan(x)*12))
    store.CompetitionOpenSinceYear = store.CompetitionOpenSinceYear.map(lambda x : int(np.nan_to_num(x) + np.isnan(x)*2100))
    store['CompetitionOpenSince'] = store.apply(lambda x : pd.datetime(x.CompetitionOpenSinceYear, x.CompetitionOpenSinceMonth, 1), axis=1)
    # и нормализуем расстояние
    store.CompetitionDistance -= store.CompetitionDistance.mean()
    store.CompetitionDistance /= store.CompetitionDistance.std()
    
    # нужно знать конкретную дату действия скидки
    store.Promo2SinceYear = store.Promo2SinceYear.map(lambda x : int(np.nan_to_num(x) + np.isnan(x)*2100))
    store.Promo2SinceWeek = store.Promo2SinceWeek.map(lambda x : int(np.nan_to_num(x) + np.isnan(x)*51))
    store['Promo2Since'] = store.apply(lambda x : (pd.datetime(x.Promo2SinceYear, 1, 1) + datetime.timedelta(7*x.Promo2SinceWeek)), axis=1)
    
    #избавляемся от ненужных данных
    store = store.drop('CompetitionOpenSinceYear', axis=1)
    store = store.drop('CompetitionOpenSinceMonth', axis=1)
    store = store.drop('Promo2SinceYear', axis=1)
    store = store.drop('Promo2SinceWeek', axis=1)
    return store

In [ ]:
store = storeinfo(store)

In [ ]:
store.head()

Количество магазинов фиксировано - ничего не мешает построить модель для каждого магазина, это позволяет и обучающая выборка, данные магазинам распределены вполне равномерно:

In [ ]:
sns.distplot(train.Store)
plt.show()

Гипотеза: Sale линейно зависит от количества Customers:

In [ ]:
sales = train.Sales
cust = train.Customers
Depend = np.array(list(zip(cust.values, sales.values)))
sns.regplot(Depend[:, 0], Depend[:, 1])
plt.show()

То есть отчетливо видно, что это прямые, коэффициент которых пробегает какой-то диапозон значений.  
Поэтому будем предугадывать среднюю цену, которую покупатель тратит в магазине, то есть  
Sale=factors(...) \* Customers  
И будем предугадывать величину factors(...).  
Теперь, если магазин закрыт, то очевидно factor = 0, то есть достаточно рассматривать открытые магазины, а искать в виде  
Sale = Open \* Customers \* OtherFactors(...)  
Будем предугадывать OtherFactors

In [ ]:
def MeanCash(df):
    df = df[df.Open == 1]
    df['MeanCash'] = np.nan_to_num(df.Sales / df.Customers)
    df = df.drop('Sales',axis=1)
    df = df.drop('Customers',axis=1)
    df = df.drop('Open',axis=1)
    return df

In [ ]:
train = MeanCash(train)

Построим распределение MeanCash

In [ ]:
sns.distplot(train.MeanCash)
plt.show()

In [ ]:
# Добавим данные о конкурентах
def Competition(df, store):
    df.Date = pd.to_datetime(df.Date)
    df['IsCompetition'] = df.apply(lambda x : int(x.Date > store.CompetitionOpenSince[x.Store]), axis=1)
    df['CompetitionDistance'] = df.apply(lambda x : store.CompetitionDistance[x.Store], axis=1)
    return df
train = Competition(train, store)

In [ ]:
train.head()